In [1]:
import pandas as pd
import sqlite3
from nltk import edit_distance as dist

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
con = sqlite3.Connection("./switrs.sqlite")

<strong style="color:red;font-size:2rem;">WARNING: This notebook will use a lot of RAM!</strong>
    
 This notebook will use about 8-10 gigs of RAM loading the two tables into dataframes.
 
 If you want to use less, make this number below larger:

In [4]:
REDUCTION_FACTOR = 10

In [5]:
collisions = pd.read_sql(
    f"SELECT * FROM collisions WHERE ABS(RANDOM() % {REDUCTION_FACTOR}) = 0", 
    con, 
    parse_dates=["collision_date"]
)

### How many collisions are there in the dataset?

In [6]:
len(collisions)

916427

### What percent of collisions involve males aged 16-25?

In [7]:
parties = pd.read_sql(f"SELECT * FROM parties WHERE ABS(RANDOM() % {REDUCTION_FACTOR}) = 0", con)

In [8]:
criteria = (
    (parties["party_sex"] == "male")
    & (parties["party_age"].between(16, 25))
)

len(parties[criteria]["case_id"].unique()) / len(parties["case_id"].unique())

0.1377241029453363

This won't match the answer in the post because we are randomly sampling.

### How many solo motorcycle crashes are there per year?

In [9]:
criteria = (
    (collisions["party_count"] == 1)  # Solo crash
    & (collisions["motorcycle_collision"] == True)  # Is motorcycle collision
)

# Select a single column to speed computation, it could be any column, but I used jurisdiction.
collisions[criteria]["jurisdiction"].groupby(collisions["collision_date"].dt.year).count()

collision_date
2001    305
2002    332
2003    379
2004    425
2005    390
2006    418
2007    484
2008    511
2009    422
2010    366
2011    385
2012    395
2013    418
2014    384
2015    438
2016    467
2017    433
2018    421
2019    380
2020    285
Name: jurisdiction, dtype: int64

### What make of vehicle has the largest fraction of accidents on the weekend? During the work week?

Only consider vehciles with at least 1,000 collisions or more.

In [10]:
collisions["day_name"] = collisions["collision_date"].dt.day_name()
collisions["is_weekend"] = collisions["day_name"].isin(("Saturday", "Sunday"))
collisions["is_weekday"] = ~collisions["is_weekend"]

merged = pd.merge(
    parties[["case_id", "vehicle_make"]], 
    collisions[["case_id", "is_weekend", "is_weekday"]], 
    how="inner",
    on=["case_id"],
)

grouped = merged[["vehicle_make", "is_weekend", "is_weekday"]].groupby("vehicle_make").sum()
grouped["total"] = grouped["is_weekend"] + grouped["is_weekday"]
selected = grouped[grouped["total"] >= 1_000].copy()

selected["weekend_ratio"] = selected["is_weekend"] / selected["total"]
selected["weekday_ratio"] = selected["is_weekday"] / selected["total"]

top_weekend = selected.sort_values(["weekend_ratio"], ascending=False).head(1)
top_weekday = selected.sort_values(["weekday_ratio"], ascending=False).head(1)

answer = pd.concat([top_weekend, top_weekday])
answer.head()

,is_weekend,is_weekday,total,weekend_ratio,weekday_ratio
vehicle_make,,,,,
NISSAN,2375.0,6590.0,8965.0,0.264919,0.735081
FREIGHTLINER,163.0,1178.0,1341.0,0.121551,0.878449


### How many different values represent "Toyota" in the Parties database? How would you go about correcting for this?

In [11]:
regex = r"(TOYOTA.*|TOY.*|TY.*)"

just_makes = parties[["vehicle_make", "party_sex"]]\
               .groupby("vehicle_make")\
               .count()\
               .reset_index()\
               .rename(columns={"party_sex": "count"})\
               .sort_values("count", ascending=False)

just_makes[just_makes["vehicle_make"].str.match(regex)]

,vehicle_make,count
1600,TOYOTA,215562
1593,TOYO,15970
1606,TOYT,14205
1599,TOYOT,277
1589,TOY,195
1611,TOYTA,23
1601,TOYOTA/,22
1612,TOYTO,9
1608,TOYT.,7
1597,TOYOA,7
